a) Go into this page with examples of students answers to a particular question. Copy some example texts and paste it into files with names like student_text_1.txt, student_text_2.txt.

In [ ]:
from pathlib import Path


path = Path.cwd().parent / "data"

with open(path/"student_text_1.txt", "r", encoding= "utf-8") as file:
    text = file.read()
    
text






'Inför allas blickar\n En av de större andledningarna att man är tal rädd är förmodligen att man är \nrädd för att folket som lyssnar på kommer göra narr av en. När man är ensam \npå en scen eller i klass rummet så är det väldigt jobbigt när det sitter ett flertal \nmänniskor som sitter och tittar på en. Jag känner mig inte så säker när jag står och \ngör en föreläsning eller en presentation. Känslan när man är framför människor \nsom man inte riktigt känner är ganska obehagligt, för man vill inte att dem ska \ntycka illa om en, eller tycka att man är konstig på något vis. Men jag tror att det \ngår att bli av med talrädslan, om man börjar i en mindre grupp och gör sig vann \nmed att prata med människor utan att bli rädd eller känna obehag.\n Peter Letmark skriver i Dagens Nyheter 2012-05-04 att det har att göra med \nevolutionära förklaringar, att man är rädd för ormar eller hundar kan ju vara \nen sak som sitter i ryggmärgen. Det kan bero på att man är rädd för att bli \nutstöt från 

b) Read these data into python and tell your LLM to grade them.

In [16]:
from google import genai


client = genai.Client()

student_anwser = path /"student_text_1.txt"

anwser = client.files.upload(file= student_anwser)


repsonse = client.models.generate_content(
    model = "gemini-2.0-flash",
    contents = ["Grade this work, output in markdown format", anwser]
)

repsonse.text

'Okay, here\'s a grade and feedback for the provided text, formatted in Markdown:\n\n**Overall Grade: C+ (Pass with Some Significant Areas for Improvement)**\n\n**General Comments:**\n\nThe text addresses the topic of fear of public speaking ("talrädsla") and presents some reasonable ideas. It connects personal experience with an external source (Peter Letmark) to support its claims. However, the writing quality needs improvement in terms of clarity, structure, and mechanics. The argument could also be more developed and nuanced.\n\n**Strengths:**\n\n*   **Relevant Topic:** The text directly addresses the assigned topic.\n*   **Personal Connection:**  The writer relates the topic to their own experiences, making it more engaging.\n*   **Source Integration:** The writer attempts to integrate an external source (Peter Letmark) to support their arguments.\n*   **Reasonable Ideas:** The ideas presented (fear of judgment, evolutionary explanations, importance of passion for the topic) are g

c) Prompt to get an output of fields proposed_grade, motivation and improvements.

In [19]:
from google import genai


client = genai.Client()

student_anwser = path /"student_text_1.txt"

anwser = client.files.upload(file= student_anwser)


repsonse = client.models.generate_content(
    model = "gemini-2.5-flash",
    contents = ["Grade this work. Output of in three points proposed_grad, motovation and improvments. Keep it shortly", anwser]
)

repsonse.text

'**proposed_grad:** Godkänt (ungefär motsvarande E/D)\n\n**motivation:** Texten behandlar ämnet på ett relevant sätt genom att blanda personliga reflektioner med en källhänvisning till Peter Letmark. Strukturen är tydlig med en inledning, egna upplevelser, en teoretisk förklaring och avslutande råd, vilket ger ett genomarbetat innehåll.\n\n**improvements:**\n1.  **Språklig korrekthet:** Arbeta med sammansatta ord (t.ex. "talrädd", "klassrummet", "jobbskäl", "självförtroende"), stavning ("anledningarna", "van vid", "utstött ur") och grammatik (t.ex. "de" istället för "dom", undvik dubbel negation som "utan att inte veta").\n2.  **Meningsbyggnad och formuleringar:** Förbättra flödet och klarheten i meningarna. Vissa meningar är långa och kan delas upp eller omformuleras för att bli lättare att följa (t.ex. referensen till Letmark).\n3.  **Styckeindelning:** Överväg att dela upp texten i fler stycken för att förbättra läsbarheten och strukturera argumenten tydligare.'

d) Now validate this with pydantic model

In [46]:
from pydantic import BaseModel, Field

class Grade(BaseModel):
    proposed_grad: str = Field(description="Proposed what grad it should be")
    motovation: str = Field(description="Motovated the grade")
    improvments: str  = Field(description="Explain what could been better")
    
    
client = genai.Client()

path = Path.cwd().parent / "data"
student_anwser = path /"student_text_1.txt"

anwser = client.files.upload(file= student_anwser)


response = client.models.generate_content(
    model = "gemini-2.5-flash",
    contents = anwser,
    config= {
        "response_mime_type": "application/json",
        "response_schema": list[Grade]
    }
    
)

response.text


'[\n  {\n    "proposed_grad": "C",\n    "motovation": "Texten behandlar ämnet talrädsla på ett personligt och relevant sätt. Den inkluderar egna reflektioner och erfarenheter, vilket skapar en bra grund. Dessutom försöker författaren att underbygga sina åsikter med en extern källa (Peter Letmark), vilket visar på en viss grad av research. Argumentationen om att tala om ämnen nära hjärtat och vikten av självförtroende är relevanta och bidrar till textens syfte. Strukturen är också hyfsat logisk, från problembeskrivning till möjliga lösningar.",\n    "improvments": "För att höja betyget skulle språket behöva förbättras avsevärt. Det finns flera stavfel och särskrivningar (t.ex. \'andledningarna\' istället för \'anledningarna\', \'tal rädd\' istället för \'talrädd\', \'klass rummet\' istället för \'klassrummet\', \'själv förtroende\' istället för \'självförtroende\', \'utstöt\' istället för \'utstött\'). Vissa formuleringar är också lite klumpiga eller repetitiva (\'att man är tal rädd är

e) Output a folder with the following txt files: proposed_grade.txt, motivation.txt and improvements.txt

In [61]:
import json 

correction_data = json.loads(response.text)

correction_data[0]


{'proposed_grad': 'C',
 'motovation': 'Texten behandlar ämnet talrädsla på ett personligt och relevant sätt. Den inkluderar egna reflektioner och erfarenheter, vilket skapar en bra grund. Dessutom försöker författaren att underbygga sina åsikter med en extern källa (Peter Letmark), vilket visar på en viss grad av research. Argumentationen om att tala om ämnen nära hjärtat och vikten av självförtroende är relevanta och bidrar till textens syfte. Strukturen är också hyfsat logisk, från problembeskrivning till möjliga lösningar.',
 'improvments': "För att höja betyget skulle språket behöva förbättras avsevärt. Det finns flera stavfel och särskrivningar (t.ex. 'andledningarna' istället för 'anledningarna', 'tal rädd' istället för 'talrädd', 'klass rummet' istället för 'klassrummet', 'själv förtroende' istället för 'självförtroende', 'utstöt' istället för 'utstött'). Vissa formuleringar är också lite klumpiga eller repetitiva ('att man är tal rädd är förmodligen att man är rädd för att folk

In [ ]:
import os


# make sure the file exists
os.makedirs("../data/grade", exist_ok=True)


# Map keys from Gemini to filename
key_to_filename = {
    "proposed_grad": "proposed_grad.txt",
    "motovation": "motovation.txt",
    "improvments": "improvments.txt"
}


# dictionary loop from the created key_to_filename
for key, filename in key_to_filename.items():
    
    value = correction_data[0].get(key, "") # create empty string if there is no data
    with open(f"../data/grade_gemini/{filename}", "w", encoding="utf-8") as file:
        file.write(value)
    

    

f) Go into skolverket for Svenska 1 and copy "Betygskriterier" for "Svenska 1". These are the criterias for the different grades. Paste this into a file called criterias.txt.

g) Now repeat b)-e) but with the criterias in your prompt as well. Can you see any differences in the outputs?

In [72]:
from pydantic import BaseModel, Field
from google import genai

path_creterias = Path.cwd().parent / "data"
path_anwser = Path.cwd().parent / "data"
    
client = genai.Client()

student_anwser = path_anwser /"student_text_1.txt"
teacher_grading = path_creterias / "criterias.txt"

anwser = client.files.upload(file= student_anwser)

grading = client.files.upload(file= teacher_grading)


class Grade(BaseModel):
    proposed_grad: str = Field(description=f"Use this to set grade: {grading}")
    motovation: str = Field(description=f"Use this to motovated the grade: {grading}")
    improvments: str  = Field(description=f"Use the to explane what can improve: {grading}")

   


response = client.models.generate_content(
    model = "gemini-2.5-flash",
    contents = anwser,
    config= {
        "response_mime_type": "application/json",
        "response_schema": list[Grade]
    }
    
)

correction = json.loads(response.text)


In [73]:
correction

[{'proposed_grad': 'B-',
  'motovation': 'Din text tar upp ett mycket relevant och igenkännbart ämne, talrädsla, på ett personligt och engagerande sätt. Du beskriver tydligt de obehagliga känslor som uppstår när man står inför en publik och kopplar det till rädslan för att bli dömd eller utstött. Det är bra att du inkluderar en referens till Peter Letmarks artikel, vilket ger en evolutionär dimension till diskussionen och visar att du har sökt information. Din slutsats om att prata om ämnen man brinner för och vikten av självförtroende är också relevanta och välformulerade.',
  'improvments': 'För att förbättra texten ytterligare skulle du kunna utveckla dina argument och lösningar mer detaljerat. När du nämner att man kan börja i en mindre grupp, skulle du kunna ge mer konkreta exempel på hur man tränar detta, eller vilka typer av övningar som kan vara effektiva. Det hade också varit intressant att fördjupa dig mer i Letmarks resonemang och kanske jämföra det med andra teorier om talr

In [ ]:


# check if map exist or else create
os.makedirs("../data/grade_teacher", exist_ok= True)


# map keys for gemini to filename
key_to_filename = {
    "proposed_grad": "proposed_grad.txt",
    "motovation": "motovation.txt",
    "improvments": "improvments.txt"
}

for key, filename in key_to_filename.items():
    
    value = correction[0].get(key)
    with open(f"../data/grade_teacher/{filename}", "w", encoding="utf-8") as file:
        
        file.write(value)






